In [1]:
import os

from src.DataReader import DataReader

In [2]:
data_directory = '../math.stackexchange.com/'

In [3]:
post_reader = DataReader(os.path.join(data_directory, 'Posts.xml'), True)
post_reader.read_data()

Reading file...
Converting...
Progress | ██████████ | 100% || Estimated time remaining: 0.0 seconds
Conversion complete...
Forming data frame...
